CP file in train and test strucuture

In [2]:
import os
import shutil
import numpy as np

def copy_images_to_train_test(source_dir, train_dir, test_dir, total_limit=4000, train_ratio=0.75):
    """
    Copie un nombre limité d'images du dossier source vers les dossiers d'entraînement (train) et de test,
    en respectant un ratio spécifié pour l'ensemble d'entraînement et une limite totale d'images.
    
    Args:
    - source_dir (str): Chemin du dossier source contenant les images.
    - train_dir (str): Chemin du dossier de destination pour les images d'entraînement.
    - test_dir (str): Chemin du dossier de destination pour les images de test.
    - total_limit (int): Limite totale du nombre d'images à copier.
    - train_ratio (float): Ratio du nombre d'images à placer dans l'ensemble d'entraînement.
    """
    
    # Liste toutes les images dans le dossier source
    all_images = [os.path.join(source_dir, img) for img in os.listdir(source_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Détermine le nombre d'images à copier basé sur la limite totale
    num_images_to_copy = min(len(all_images), total_limit)
    
    # Mélange aléatoirement les images et sélectionne un sous-ensemble basé sur la limite totale
    np.random.shuffle(all_images)
    selected_images = all_images[:num_images_to_copy]
    
    # Calcule le nombre d'images pour l'ensemble d'entraînement
    num_train = int(len(selected_images) * train_ratio)
    
    # Sépare les images sélectionnées en ensembles d'entraînement et de test
    train_images = selected_images[:num_train]
    test_images = selected_images[num_train:]
    
    # Crée les dossiers s'ils n'existent pas
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)
    
    # Copie les images d'entraînement
    for image_path in train_images:
        dest_path = os.path.join(train_dir, os.path.basename(image_path))
        shutil.copy(image_path, dest_path)
        print(f"Image copiée vers {dest_path} pour l'entraînement.")
    
    # Copie les images de test
    for image_path in test_images:
        dest_path = os.path.join(test_dir, os.path.basename(image_path))
        shutil.copy(image_path, dest_path)
        print(f"Image copiée vers {dest_path} pour les tests.")


In [ ]:
# Exemple d'utilisation
source_directory = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/dataset-full/Martensite_microstructure_img_cropped_V2_yolo'
train_directory = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/train'
test_directory = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/test/test'
copy_images_to_train_test(source_directory, train_directory, test_directory)

Fonction V2, Répartit un ensemble de données en images d'entraînement, de validation et de test

In [1]:
import os
import shutil
import numpy as np

def split_dataset_into_train_val_test(class_dirs, output_base_dir, ratios=(0.7, 0.2, 0.1)):
    """
    Répartit un ensemble de données en images d'entraînement, de validation et de test,
    en respectant les ratios spécifiés pour chaque ensemble.
    
    Args:
    - class_dirs (list): Liste contenant les chemins vers les dossiers de chaque classe.
    - output_base_dir (str): Chemin de base pour les dossiers d'entraînement, de validation et de test.
    - ratios (tuple): Ratios de répartition pour les ensembles d'entraînement, de validation et de test.
    """
    
    assert 0.99 <= sum(ratios) <= 1.01, "Les ratios doivent sommer à environ 1"
    
    # Ajoutez ici tous les formats d'image que vous souhaitez inclure
    valid_extensions = ('.bmp', '.jpeg', '.png', '.webp', '.jpg')
    
    for class_dir in class_dirs:
        class_name = os.path.basename(class_dir)
        all_images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.lower().endswith(valid_extensions)]
        np.random.shuffle(all_images)
        
        train_end = int(len(all_images) * ratios[0])
        val_end = train_end + int(len(all_images) * ratios[1])
        
        train_images = all_images[:train_end]
        val_images = all_images[train_end:val_end]
        test_images = all_images[val_end:]
        
        for image_list, subdir in zip([train_images, val_images, test_images], ['train', 'val', 'test']):
            dest_dir = os.path.join(output_base_dir, subdir, class_name)
            os.makedirs(dest_dir, exist_ok=True)
            for image_path in image_list:
                shutil.copy(image_path, dest_dir)
                print(f"Image {os.path.basename(image_path)} copiée vers {dest_dir}.")

In [ ]:
# Exemple d'utilisation
class_dirs = ['/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/dataset-full/Martensite_microstructure_img_cropped_V3_manualy_cleaned_yolo',
              '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/dataset-full/pearlite_microstructure_img_cropped_V3_cleaned_yolo']

output_base_dir = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_825_cleaned'

ratios = (0.75, 0.2, 0.05)  # Par exemple : 75% entraînement, 20% validation, 5% test

split_dataset_into_train_val_test(class_dirs, output_base_dir, ratios)

Fonction V3, Répartit un ensemble de données en images d'entraînement, de validation et de test, avec limite d'images

In [5]:
import os
import shutil
import numpy as np

def split_dataset_into_train_val_test(class_dirs, output_base_dir, ratios=(0.7, 0.2, 0.1), total_limit=None):
    """
    Répartit un ensemble de données en images d'entraînement, de validation et de test,
    en respectant les ratios spécifiés pour chaque ensemble et une limite totale d'images optionnelle.
    
    Args:
    - class_dirs (list): Liste contenant les chemins vers les dossiers de chaque classe.
    - output_base_dir (str): Chemin de base pour les dossiers d'entraînement, de validation et de test.
    - ratios (tuple): Ratios de répartition pour les ensembles d'entraînement, de validation et de test.
    - total_limit (int, optional): Limite totale du nombre d'images à inclure. Si None, toutes les images sont incluses.
    """
    
    assert 0.99 <= sum(ratios) <= 1.01, "Les ratios doivent sommer à environ 1"
    
    # Ajoutez ici tous les formats d'image que vous souhaitez inclure
    valid_extensions = ('.bmp', '.jpeg', '.png', '.webp', '.jpg')
    
    for class_dir in class_dirs:
        class_name = os.path.basename(class_dir)
        all_images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.lower().endswith(valid_extensions)]
        
        # Applique la limite totale si spécifiée
        if total_limit is not None:
            all_images = all_images[:min(total_limit, len(all_images))]
        
        np.random.shuffle(all_images)
        
        train_end = int(len(all_images) * ratios[0])
        val_end = train_end + int(len(all_images) * ratios[1])
        
        train_images = all_images[:train_end]
        val_images = all_images[train_end:val_end]
        test_images = all_images[val_end:]
        
        for image_list, subdir in zip([train_images, val_images, test_images], ['train', 'val', 'test']):
            dest_dir = os.path.join(output_base_dir, subdir, class_name)
            os.makedirs(dest_dir, exist_ok=True)
            for image_path in image_list:
                shutil.copy(image_path, dest_dir)
                print(f"Image {os.path.basename(image_path)} copiée vers {dest_dir}.")


In [6]:
# Exemple d'utilisation
class_dirs = ['/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/full/Martensite',
              '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/full/Pearlite']

output_base_dir = '/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img'

ratios = (0.75, 0.2, 0.05)  # Par exemple : 75% entraînement, 20% validation, 5% test

split_dataset_into_train_val_test(class_dirs, output_base_dir, ratios, total_limit=4000)

Image right_a_microstructure_of_martensite_cropped_0.png copiée vers /home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/Martensite.
Image images - 2024-03-16T151055.145_cropped_4.png copiée vers /home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/Martensite.
Image images - 2024-03-16T151055.405_cropped_2.png copiée vers /home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/Martensite.
Image images - 2024-03-16T151042.007_cropped_0.png copiée vers /home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/Martensite.
Image images - 2024-03-16T151054.291_cropped_3.png copiée vers /home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/train/Martensite.
Image 3-s2.0-B9780857092380500061-f06-05-9780857092380_cropped_0.png 

Check if data wrong

In [1]:
from PIL import Image
import os

def check_images(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            try:
                img_path = os.path.join(directory, filename)
                with Image.open(img_path) as img:
                    img.verify()  # Vérifie si l'image est corrompue
            except (IOError, SyntaxError) as e:
                print(f'Bad file: {filename}')  # Signale le fichier corrompu

In [4]:
# Appliquer la fonction à votre dossier d'images
check_images('/home/riccardo/Visual_Studio_Code/Grain-segmentation_prjt/Dataset_Structures_modes/ImageDataGen_1300img/full/Pearlite')